In [22]:
# Importing required libraries and dependencies 
import pandas as pd
import datetime as dt 
import numpy as np
from prophet import Prophet 
import matplotlib.pyplot as plt
import hvplot.pandas as hvplot


In [25]:
private_sector_df = pd.read_csv("../Resources/privatesector_Data.csv", delimiter=';').dropna()
display(private_sector_df.head())
display(private_sector_df.tail())
display(private_sector_df.dtypes)
display(private_sector_df.describe())


,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
0,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,Brazil,BRA,1970.0,1.189510e+01
2,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,India,IND,1970.0,5.556494e+00
5,Agricultural raw materials imports (% of merch...,TM.VAL.AGRI.ZS.UN,Brazil,BRA,1970.0,1.851044e+00
7,Agricultural raw materials imports (% of merch...,TM.VAL.AGRI.ZS.UN,India,IND,1970.0,9.207277e+00
64,Commercial service exports (current US$),TX.VAL.SERV.CD.WT,South Africa,ZAF,1970.0,5.739998e+08


,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
42236,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,China,CHN,2019.0,50.738701
42237,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,India,IND,2019.0,17.704861
42238,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,Russian Federation,RUS,2019.0,37.015007
42239,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,South Africa,ZAF,2019.0,20.529111
42243,Value lost due to electrical outages (% of sal...,IC.FRM.OUTG.ZS,Russian Federation,RUS,2019.0,0.300000


SeriesName      object
SeriesCode      object
CountryName     object
CountryCode     object
Year           float64
Value          float64
dtype: object

,Year,Value
count,17420.000000,1.742000e+04
mean,2001.752411,1.159949e+10
std,12.594159,1.057388e+11
min,1970.000000,0.000000e+00
25%,1994.000000,4.474096e+00
50%,2004.000000,1.660000e+01
75%,2012.000000,8.529821e+01
max,2019.000000,2.501330e+12


In [27]:
#Fetch Data for the last 10 years
private_sector_df = private_sector_df.loc[private_sector_df['Year'] >= 2009]
display(private_sector_df.head())
display(private_sector_df.tail())
display(private_sector_df.describe())

,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
32955,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,Brazil,BRA,2009.0,3.770854
32956,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,China,CHN,2009.0,0.448398
32957,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,India,IND,2009.0,1.164823
32958,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,Russian Federation,RUS,2009.0,2.301575
32959,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,South Africa,ZAF,2009.0,1.922157


,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
42236,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,China,CHN,2019.0,50.738701
42237,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,India,IND,2019.0,17.704861
42238,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,Russian Federation,RUS,2019.0,37.015007
42239,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,South Africa,ZAF,2019.0,20.529111
42243,Value lost due to electrical outages (% of sal...,IC.FRM.OUTG.ZS,Russian Federation,RUS,2019.0,0.300000


,Year,Value
count,6463.000000,6.463000e+03
mean,2014.069782,2.184586e+10
std,3.007304,1.624208e+11
min,2009.000000,0.000000e+00
25%,2012.000000,4.939012e+00
50%,2014.000000,1.731051e+01
75%,2017.000000,1.000000e+02
max,2019.000000,2.501330e+12
